In [2]:
import csv
import cv2
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0, EfficientNetB3, EfficientNetB2, EfficientNetB4
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle
import math
import pandas as pd
from keras_preprocessing.image import ImageDataGenerator
import json

In [3]:
#Mount google drive to colab
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
images_path = "./gdrive/MyDrive/lumen/data/"
annotations_path = "./gdrive/MyDrive/lumen/processed_annotations.csv"
train_annotations_path = "./gdrive/MyDrive/lumen/train_annotations.csv"
validation_annotations_path = "./gdrive/MyDrive/lumen/validation_annotations.csv"
test_annotations_path = "./gdrive/MyDrive/lumen/test_annotations.csv"
model_path = "./gdrive/MyDrive/models/region_guessr_eff_net_mdl_wts.hdf5" 
IMAGE_SIZE = 128           ##300 for B3, 224 za B0 https://keras.io/examples/vision/image_classification_efficientnet_fine_tuning/
BATCH_SIZE = 128
regions_to_labels = {"Istra": 0, "Lika": 1, "Dalmacija": 2, "Sjever": 3, "Slavonija": 4, "Sredisnja": 5}

In [5]:
def build_model():
  inputs = layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
  # model = EfficientNetB4(include_top=False, input_tensor=inputs, weights="imagenet")
  # model.trainable = False
  model = EfficientNetB4(include_top=False, input_tensor=inputs, weights="imagenet")
  x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
  x = layers.BatchNormalization()(x)
  top_dropout_rate = 0.1
  x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
  outputs = layers.Dense(6, activation="softmax", name="pred")(x)
  model = tf.keras.Model(inputs, outputs, name="EfficientNet")
  optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
  model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
  return model

In [6]:
train_records_path = "./gdrive/MyDrive/tf_records/train"
validation_records_path = "./gdrive/MyDrive/tf_records/validation"

In [7]:
TRAIN_FILENAMES = tf.io.gfile.glob(train_records_path + "/*.tfrecords")
print(train_records_path + "/train*.tfrec")
print("Train TFRecord Files:", len(TRAIN_FILENAMES))

VAL_FILENAMES = tf.io.gfile.glob(validation_records_path + "/*.tfrecords")
print(validation_records_path + "/validation*.tfrec")
print("Train TFRecord Files:", len(VAL_FILENAMES))

AUTOTUNE = tf.data.AUTOTUNE

./gdrive/MyDrive/tf_records/train/train*.tfrec
Train TFRecord Files: 5
./gdrive/MyDrive/tf_records/validation/validation*.tfrec
Train TFRecord Files: 1


In [8]:
def parse_tfr_element(element):
  #use the same structure as above; it's kinda an outline of the structure we now want to create
  data = {
      # 'height': tf.io.FixedLenFeature([], tf.int64),
      # 'width':tf.io.FixedLenFeature([], tf.int64),
      'label':tf.io.FixedLenFeature([], tf.int64),
      'raw_image' : tf.io.FixedLenFeature([], tf.string),
    }

    
  content = tf.io.parse_single_example(element, data)
  
  # height = content['height']
  # width = content['width']
  label = content['label']
  raw_image = content['raw_image']
  
  
  #get our 'feature'-- our image -- and reshape it appropriately
  feature = tf.io.parse_tensor(raw_image, out_type=tf.uint8)
  feature = tf.reshape(feature, shape=[IMAGE_SIZE, IMAGE_SIZE,3])
  return (feature, label)

In [9]:
def decode_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.cast(image, tf.float32)
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    return image

In [10]:
def read_tfrecord(example, labeled):
    tfrecord_format = (
        {
          'label':tf.io.FixedLenFeature([], tf.int64),
          'raw_image' : tf.io.FixedLenFeature([], tf.string),
        }
        if labeled
        else {"raw_image": tf.io.FixedLenFeature([], tf.string),}
    )
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example["raw_image"])
    if labeled:
        label = tf.cast(example["label"], tf.int32)
        return image, label
    return image

In [11]:
def load_dataset(filenames, labeled=True):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False  # disable order, increase speed
    dataset = tf.data.TFRecordDataset(
        filenames
    )  # automatically interleaves reads from multiple files
    dataset = dataset.with_options(
        ignore_order
    )  # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(
        parse_tfr_element, num_parallel_calls=AUTOTUNE
    )
    # returns a dataset of (image, label) pairs if labeled=True or just images if labeled=False
    return dataset

In [12]:
def get_dataset(filenames, labeled=True):
    dataset = load_dataset(filenames, labeled=labeled)
    dataset = dataset.shuffle(2048)
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

In [13]:
train_dataset = get_dataset(TRAIN_FILENAMES)
validation_dataset = get_dataset(VAL_FILENAMES)

In [14]:
model = build_model()
epochs = 25
model_checkpoint_callback = ModelCheckpoint(
    filepath=model_path,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)
# model.load_weights(model_path)  ##Ukljuciti ako zelimo nastaviti od zadnjeg treninga, iskljuciti ako zelimo poceti od nule
# model.fit(x=x, y=y, batch_size=BATCH_SIZE, epochs=epochs, validation_split=0.1, verbose=1, callbacks=[model_checkpoint_callback], shuffle=True)
# model.fit(train_dataset, batch_size=BATCH_SIZE, epochs=epochs, validation_data=validation_dataset, verbose=1, callbacks=[model_checkpoint_callback], shuffle=True)

71696384/71686520 [==============================] - 1s 0us/step


In [15]:
# model.load_weights(model_path)  ##Ukljuciti ako zelimo nastaviti od zadnjeg treninga, iskljuciti ako zelimo poceti od nule
# model.evaluate(x=x_test, y=y_test)

In [16]:
# {"Istra": 0, "Lika": 1, "Dalmacija": 2, "Sjever": 3, "Slavonija": 4, "Sredisnja": 5}

# predicted = model.predict(x_test)

# print(predicted[50:100])
# print(y_test[50:100])

In [17]:
def get_coords(predicted, ground):
  # istra_pred = []
  # lika_pred = []
  # dalmacija_pred = []
  # sjever_pred = []
  # slavonija_pred = []
  # sredisnja_pred = []

  istra = []
  lika = []
  dalmacija = []
  sjever = []
  slavonija = []
  sredisnja = []

  istra_center = (0.44101925966301925, 0.07265188518324361)       ##(45.220319, 14.032990)
  lika_center = (0.23864939356300255, 0.24680208841686116)        ##(44.703375, 15.422761)
  dalmacija_center = (0.10960616898135371, 0.4855092079983375)    ##(43.845979, 16.164338)
  sjever_center = (0.933957310114781, 0.545529198432239)          ##(46.217436, 16.362092)
  slavonija_center = (0.643344175128224, 0.8253488420304437)      ##(45.482822, 17.872712)
  sredisnja_center = (0.7078991227853865, 0.5472032808002736)     ##(45.609774, 16.367585)

  # RMSE = np.sqrt(np.square(np.subtract(predicted, ground)))
  coords = []
  for i in range(len(predicted)):
    if np.argmax(predicted[i]) == 0:
      istra.append(math.sqrt((ground[i][0] - istra_center[0])**2 + (ground[i][1] - istra_center[1])**2))
      coords.append(istra_center)
    elif np.argmax(predicted[i]) == 1:
      lika.append(math.sqrt((ground[i][0] - lika_center[0])**2 + (ground[i][1] - lika_center[1])**2))
      coords.append(lika_center)
    elif np.argmax(predicted[i]) == 2:
      dalmacija.append(math.sqrt((ground[i][0] - dalmacija_center[0])**2 + (ground[i][1] - dalmacija_center[1])**2))
      coords.append(dalmacija_center)
    elif np.argmax(predicted[i]) == 3:
      sjever.append(math.sqrt((ground[i][0] - sjever_center[0])**2 + (ground[i][1] - sjever_center[1])**2))
      coords.append(sjever_center)
    elif np.argmax(predicted[i]) == 4:
      slavonija.append(math.sqrt((ground[i][0] - slavonija_center[0])**2 + (ground[i][1] - slavonija_center[1])**2))
      coords.append(slavonija_center)
    elif np.argmax(predicted[i]) == 5:
      sredisnja.append(math.sqrt((ground[i][0] - sredisnja_center[0])**2 + (ground[i][1] - sredisnja_center[1])**2))
      coords.append(sredisnja_center)
  print(np.mean(istra))
  print(np.mean(lika))
  print(np.mean(dalmacija))
  print(np.mean(sjever))
  print(np.mean(slavonija))
  print(np.mean(sredisnja))
  print("sredisnja: ", sredisnja)
  MSE = np.mean(np.square(np.subtract(coords, ground)))
  print("MSE:", MSE)

In [18]:
def get_reg_data(annotations_path, pickle_path_pred, pickle_path_coords, start=0):
  if start == 0:
    predictions = []
    coords = []
  else:
    store_pred = open(pickle_path_pred, "rb")
    predictions = pickle.load(store_pred)
    store_pred.close()
    store_ground = open(pickle_path_coords, "rb")
    coords = pickle.load(store_ground)
    store_ground.close()
    start = len(predictions)
  csv_reader = csv.reader(open(annotations_path, 'r'))
  next(csv_reader)
  i = start
  for row in csv_reader:
    if i % 500 == 0:
      print(i)
      store_pred = open(pickle_path_pred, "wb")
      pickle.dump(predictions, store_pred)
      store_pred.close()
      store_ground = open(pickle_path_coords, "wb")
      pickle.dump(coords, store_ground)
      store_ground.close()
    try:
      image = cv2.imread(images_path + row[0]) 
      image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
      coords.append((float(row[5]), float(row[4])))
      image_list = []
      image_list.append(image)
      image_list = np.array(image_list)
      predictions.append(np.array(model(image_list)))
      i += 1
    except:
      print(images_path + row[0])
  return np.array(predictions).reshape(-1, 6), np.array(coords)

In [19]:
def read_reg_data(pickle_path_pred, pickle_path_coords):
  store_preds = open(pickle_path_pred, "rb")
  preds = pickle.load(store_preds)
  store_preds.close()

  store_coords = open(pickle_path_coords, "rb")
  coords = pickle.load(store_coords)
  store_coords.close()
  return np.reshape(np.array(preds), (-1, 6)), np.array(coords)

In [20]:
model.load_weights(model_path)

train_path = "./gdrive/MyDrive/lumen/regression_train_annotations.csv"
validation_path = "./gdrive/MyDrive/lumen/regression_validation_annotations.csv"

pickle_train_path_pred = "./gdrive/MyDrive/lumen/pickle/predictions_effnet_train.pkl"
pickle_train_path_coords = "./gdrive/MyDrive/lumen/pickle/coords_train.pkl"

pickle_val_path_pred = "./gdrive/MyDrive/lumen/pickle/predictions_effnet_val.pkl"
pickle_val_path_coords = "./gdrive/MyDrive/lumen/pickle/coords_val.pkl"

In [21]:
# train_predictions, train_coords = get_reg_data(train_path, pickle_train_path_pred, pickle_train_path_coords, 0)
# val_predictions, val_coords = get_reg_data(validation_path, pickle_val_path_pred, pickle_val_path_coords, 0)
# # model.evaluate(validation_dataset)
# labels_summary = [0 for i in range(6)]
# for i in train_predictions:
#   labels_summary[np.argmax(i)] += 1
# print(labels_summary)
# get_coords(train_predictions, train_coords)
# get_coords(val_predictions, val_coords)

In [22]:
def build_regression_model():
  inputs = layers.Input(shape=(6,))
  x = layers.Dense(128, activation="relu", name="hidden_layer")(inputs)
  outputs = layers.Dense(2, activation="sigmoid", name="pred")(x)
  model = tf.keras.Model(inputs, outputs, name="EfficientNet")
  optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
  model.compile(optimizer=optimizer, loss="mean_squared_error", metrics=["mean_squared_error"])
  return model

In [23]:
reg_model = build_regression_model()
reg_model_path = "./gdrive/MyDrive/models/region_guessr_eff_net_small_reg_mdl_wts.hdf5"
reg_model_checkpoint_callback = ModelCheckpoint(
    filepath=reg_model_path,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)
# reg_model.fit(x=train_predictions, y=train_coords, validation_data=(val_predictions, val_coords), epochs=epochs, verbose=1, callbacks=[reg_model_checkpoint_callback], shuffle=True)

In [24]:
reg_model.load_weights(reg_model_path)
train_preds, train_coords = read_reg_data(pickle_train_path_pred, pickle_train_path_coords)
val_preds, val_coords = read_reg_data(pickle_val_path_pred, pickle_val_path_coords)
# reg_model.evaluate(x=train_preds, y=train_coords)
# reg_model.evaluate(x=val_preds, y=val_coords)

In [25]:
def visualise_sample(pred, ground, map_path):
  map = cv2.imread(map_path)
  plt.figure()
  plt.imshow(cv2.flip(map, 0))

  current_axis = plt.gca()
  plt.gca().invert_yaxis()

  x0 = pred[0] * map.shape[0]
  y0 = pred[1] * map.shape[1]
  current_axis.add_patch(plt.Circle((x0, y0), 20, color="red", fill=True, linewidth=2))

  x0 = ground[0] * map.shape[0]
  y0 = ground[1] * map.shape[1]
  current_axis.add_patch(plt.Circle((x0, y0), 20, color="blue", fill=True, linewidth=2))

In [26]:
def transform(json_x_path, json_y_path, data):
  with open(json_x_path, 'r') as file:
    transformation_x = json.load(file)
  with open(json_y_path, 'r') as file:
    transformation_y = json.load(file)
  transformed_data = []
  for coords in data:
    transformed = []
    for trans in transformation_x:
      if coords[0] <= trans["target_range_end"]:
        range_perc = (coords[0] - trans["target_range_start"])/(trans["target_range_end"] - trans["target_range_start"])
        transformed.append(range_perc * (trans["original_range_end"] - trans["original_range_start"]) + trans["original_range_start"])
        if range_perc * (trans["original_range_end"] - trans["original_range_start"]) + trans["original_range_start"] > 1:
          print("Error: x is bigger then 1")
        break
    for trans in transformation_y:
      if coords[1] <= trans["target_range_end"]:
        range_perc = (coords[1] - trans["target_range_start"])/(trans["target_range_end"] - trans["target_range_start"])
        transformed.append(range_perc * (trans["original_range_end"] - trans["original_range_start"]) + trans["original_range_start"])
        if range_perc * (trans["original_range_end"] - trans["original_range_start"]) + trans["original_range_start"] > 1:
          print("Error: y is bigger then 1")
        break
    transformed_data.append(transformed)
  return transformed_data

In [34]:
json_x_path = "./gdrive/MyDrive/lumen/transforms/x.json"
json_y_path = "./gdrive/MyDrive/lumen/transforms/y.json"
map_path = "./gdrive/MyDrive/lumen/croatia-satellite-map.jpg"

sample_predictions = reg_model(val_preds)
sample_predictions = transform(json_x_path=json_x_path, json_y_path=json_y_path, data=sample_predictions)
sample_coords = transform(json_x_path=json_x_path, json_y_path=json_y_path, data=val_coords)

print(np.argwhere(val_coords == [0.6836538,  0.8009546 ]))
for i in range(100, 100 + len(sample_predictions[:80]), 4):
  visualise_sample(pred=sample_predictions[i], ground=sample_coords[i], map_path=map_path)

Output hidden; open in https://colab.research.google.com to view.